In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!git clone https://github.com/valkryhx/chatGLM-6B-QLoRA

fatal: destination path 'chatGLM-6B-QLoRA' already exists and is not an empty directory.


In [3]:
%cd chatGLM-6B-QLoRA
!git pull --all --force 
!pip install -r requirements.txt

/kaggle/working/chatGLM-6B-QLoRA
Fetching origin
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 750 bytes | 150.00 KiB/s, done.
From https://github.com/valkryhx/chatGLM-6B-QLoRA
   8d61438..b1dfef7  main       -> origin/main
Updating 8d61438..b1dfef7
Fast-forward
 rm_trl.py | 39 ++++++++++++++++++++++-----------------
 1 file changed, 22 insertions(+), 17 deletions(-)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 12.9 MB/s eta 0:00:0000:0100:01
  

# 

In [5]:
!git pull --all --force
!CUDA_VISIBLE_DEVICES=0 python  rm_trl.py \
--model_name 'THUDM/chatglm2-6b' \
--resume_from_checkpoint /kaggle/working/chatGLM-6B-QLoRA/reward_model_0809_v1/checkpoint-50 \
--num_train_epochs 2 \
--gradient_accumulation_steps 1 \
--per_device_train_batch_size 4 \
--per_device_eval_batch_size  4 \
--max_length 512 \
--output_dir ./reward_model_0810_v2 \
--train_subset 80 \
--eval_subset 20 \
--local_rank 0  \
--bf16 False

Fetching origin
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 674 bytes | 134.00 KiB/s, done.
From https://github.com/valkryhx/chatGLM-6B-QLoRA
   b1dfef7..808ddf3  main       -> origin/main
Updating b1dfef7..808ddf3
Fast-forward
 rm_trl.py | 1 +
 1 file changed, 1 insertion(+)
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']

# 上面的warning There were missing keys in the checkpoint model loaded: 'transformer.embedding.word_embeddings.weight', 'transformer.rotary_pos_emb.inv_freq', 'transformer.encoder.layers.0.input_layernorm.weight', ....
# 是 trainingArguments中设置 load_best_model_at_end = True 后出现的
# 说明 trainer的确在最后把最好的那个adapters的参数做了load（只不过load时strict=True要求严格匹配了）
# 这说明保存在output_dir中的pytorch_model.bin和vhead.bin参数都是最佳的 我们来验证下

验证成功  跑的是py中的test_load_best()
